In [1]:
import datasets
from tracker import ClientTracker, PunishedClientPool, make_client
import numpy as np
import numpy as np
import flwr as fl
from federated_learning import CustomFedAvgWithDetection
from torch.utils.data import Subset
from torchvision import datasets, transforms

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-18 11:01:24,877	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# --- 학습 시작 ---
def start_federated_learning(penalty_threshold=3, penalty_mode: str = "accumulated", switch_round=15, consecutive_mode=False, decay=5):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    full_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    partitions = [Subset(full_data, idx) for idx in np.array_split(np.random.permutation(len(full_data)), 10)]
    tracker = ClientTracker()

    labels = ["normal"] * 5 + ["lazy", "random", "echo", "small", "normal"]
    clients = [make_client(i, partitions[i], labels[i]) for i in range(10)]
    assert len(labels) == len(partitions), "라벨 수가 클라이언트 수와 일치하지 않습니다."

    pool = PunishedClientPool(clients, partitions, consecutive_mode=consecutive_mode)

    fl.simulation.start_simulation(
        client_fn=pool.get_client_fn(),
        num_clients=10,
        config=fl.server.ServerConfig(num_rounds=30),
        strategy = CustomFedAvgWithDetection(
            pool,
            tracker=tracker,
            penalty_threshold=penalty_threshold,  # 또는 6 으로 바꿔가며 실험
            penalty_mode=penalty_mode,
            switch_round=switch_round,
            eval_dataset=full_data,
            decay=decay
        )
    )
    # 시뮬레이션 종료 후 CSV 저장 및 시각화
    tracker.save_global_loss_csv(f"fl_global_loss_log_{penalty_mode}_{penalty_threshold}_{switch_round}_decay_{decay}_min_test2.csv")  # 🔵 loss 기록
    tracker.plot_loss_and_client_status()   # loss & cilent 비율 그래프 합치기
    tracker.save_distribution_csv(f"fl_distribution_log_{penalty_mode}_{penalty_threshold}_{switch_round}_decay_{decay}_min_test2.csv")
    tracker.plot_clss_over_rounds()   # C-LSS 그래프
    

    return tracker  # 마지막에 반드시 추가

In [3]:
import ray
ray.shutdown()